In [1]:
#|default_exp maximdataset


In [2]:
#| export
import polars as pl
import pandas as pd
import gc
import os
import numpy as np
import torch
from typing import Iterator, Optional, Sized
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import polars as pl
import pandas as pd
import os,gc
import numpy as np
from collections import OrderedDict
from sklearn.preprocessing import RobustScaler
from scipy.interpolate import interp1d
from fastai.vision.all import DataLoaders, OptimWrapper
from torch_geometric.nn.pool import knn_graph
import torch.nn.functional as F
from torch_geometric.utils import to_dense_adj


/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


In [3]:
import numpy as np

def generate_mask(L):
    mask = np.random.choice([True, False], size=L, p=[0.93, 0.07])
    return mask

In [4]:
import sys
sys.path.append('/opt/slh/icecube/')
from icecube.models import EncoderWithDirectionReconstructionV10, EncoderWithDirectionReconstructionV11, EncoderWithDirectionReconstructionV13, EncoderWithDirectionReconstructionV12_V2, EncoderWithDirectionReconstructionV14, EncoderWithDirectionReconstructionV20, EncoderWithDirectionReconstructionV22, OrganizerBaseline
#from icecube.modelsgraph import PytorchEGNNV0, 
import matplotlib.pyplot as plt
from icecube.data_train_v3 import RandomChunkSampler,LenMatchBatchSampler,IceCubeCache, DeviceDataLoader

graphnet: INFO     2023-04-20 17:40:33 - get_logger - Writing log to logs/graphnet_20230420-174033.log


In [5]:
bs = 32
SELECTION = 'total'
NUM_WORKERS = 4
PATH = '../data/'
L = 192

ds_train = IceCubeCache(PATH, mode='train', L=L, selection=SELECTION,reduce_size=0.125)
ds_train_len = IceCubeCache(PATH, mode='train', L=L, reduce_size=0.125, selection=SELECTION, mask_only=True)
sampler_train = RandomChunkSampler(ds_train_len, chunks=ds_train.chunks)
len_sampler_train = LenMatchBatchSampler(sampler_train, batch_size=bs, drop_last=True)
dl_train = DeviceDataLoader(torch.utils.data.DataLoader(ds_train, 
            batch_sampler=len_sampler_train, num_workers=NUM_WORKERS, persistent_workers=True))

ds_val = IceCubeCache(PATH, mode='eval', L=L, selection=SELECTION)
ds_val_len = IceCubeCache(PATH, mode='eval', L=L, selection=SELECTION, mask_only=True)
sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=bs, drop_last=False)
dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, batch_sampler=len_sampler_val,
            num_workers=0))


dls = DataLoaders(dl_train,dl_val)

In [6]:
x0 = next(iter(dls[0]))[0]

/opt/slh/icecube/icecube/data_train_v3.py:264: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  L = max(1,L // 16)


In [7]:
md = OrganizerBaseline().eval().cuda()
with torch.no_grad():
    out = md(x0)

In [8]:
out.shape

torch.Size([32, 3])

In [ ]:
# mask = batch["mask"]
# Lmax = mask.sum(-1).max()
# fe = ExtractorV0(dim=256, dim_base=96).cuda()
# x = fe(batch, Lmax)
# mask = mask[:,:Lmax]
# #ptr = torch.cat([torch.zeros(1, dtype=torch.long, device=batch.device), mask.sum(1).cumsum(0)])
# x = x[mask]
# batch_index = mask.nonzero()[:, 0]
# pos = batch['pos'][:,:Lmax][mask]
# edge_index = knn_graph(
#                 x=pos,
#                 k=8,
#                 batch=batch_index,
#             ).to(mask.device)


In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()

In [ ]:
generate_mask(100).sum()

In [ ]:
import timm


In [ ]:
out

tensor([[-0.5899,  1.1641,  0.4085, -0.1833]])

In [ ]:
F.interpolate(out[0], size=(128,128), mode='bilinear', align_corners=False).shape

torch.Size([1, 64, 128, 128])

In [ ]:
adaptive_avgmax_pool2d(out[0])

torch.Size([1, 512])

In [2]:
import torch


tensor([0.2150, 0.2150, 0.2150, 0.2150])

In [4]:
torch.tensor([0.215, 0.215, 0.215, 0.215]).sum()

tensor(0.8600)

In [13]:
(torch.tensor([0.0175, 0.0175, 0.0175, 0.0175, 0.0700]) * 0.7).sum()

tensor(0.0980)

In [10]:
torch.tensor([0.0158, 0.0158, 0.0158, 0.0158, 0.0630]).sum()

tensor(0.1262)